In [35]:
import cv2
import numpy as np
from ultralytics import YOLO
import os
import glob
import csv

# Load YOLOv8 model
model = YOLO('assignment_3_model_yolo11s.pt')

# Root folder
#base_folder = r"F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\captured_images2\Scenario_3\Place_1\Object_100\Distance_d11"
#base_folder = r"F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\captured_images2\Scenario_2\Place_1\Object_100\Distance_d11"
base_folder = r"F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\captured_images2\Scenario_1\Place_1\Object_100\Distance_d11"


# Subfolder paths
rgb_folder = os.path.join(base_folder, "RGB")
npy_folder = os.path.join(base_folder, "NPY")
csv_output_path = os.path.join(base_folder, "object_distances.csv")

# RealSense D455: convert mm to meters
depth_scale = 0.001

# Target object names (CSV column headers)
expected_classes = [
    "apple", "watch", "airpods", "card", "mouse",
    "plug", "stapler", "orange", "phone", "stickypad"
]

# Mapping from YOLO class names to expected CSV column names
class_name_map = {
    "Apple": "apple",
    "Watch": "watch",
    "AirPod": "airpods",
    "CardHolder": "card",
    "Mouse": "mouse",
    "Key": "plug",
    "Stapler": "stapler",
    "Orange": "orange",
    "Mobile": "phone",
    "StickyNote": "stickypad"
}

# List all RGB files
rgb_files = sorted(glob.glob(os.path.join(rgb_folder, "rgb_frame_*.png")))

# Write to CSV
with open(csv_output_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["image"] + expected_classes)

    for rgb_path in rgb_files:
        index = os.path.splitext(os.path.basename(rgb_path))[0].split("_")[-1]
        depth_path = os.path.join(npy_folder, f"depth_raw_{index}.npy")

        if not os.path.exists(depth_path):
            print(f"[Warning] Missing depth file for: {rgb_path}")
            continue

        # Load images
        rgb_image = cv2.imread(rgb_path)
        depth_image = np.load(depth_path).astype(np.float32) * depth_scale

        # Run object detection
        results = model(rgb_image)
        object_depths = {cls: "" for cls in expected_classes}

        for result in results:
            boxes = result.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                confidence = box.conf[0].cpu().numpy()
                class_id = int(box.cls[0].cpu().numpy())
                detected_name = model.names[class_id]
                class_name = class_name_map.get(detected_name, None)

                if confidence < 0.5 or class_name is None:
                    continue

                # Get valid depth pixels
                crop = depth_image[y1:y2, x1:x2]
                valid_depths = crop[(crop > 0) & (crop < 10)]
                if valid_depths.size == 0:
                    continue

                # Store median depth
                object_depths[class_name] = round(np.median(valid_depths*0.92), 3)

        # Write one row to CSV
        row = [os.path.basename(rgb_path)] + [object_depths[cls] for cls in expected_classes]
        writer.writerow(row)

print(f"✅ CSV saved: {csv_output_path}")



0: 480x640 1 AirPod, 1 Apple, 1 CardHolder, 1 Key, 1 Mobile, 1 Orange, 1 Stapler, 1 StickyNote, 1 Watch, 145.6ms
Speed: 4.4ms preprocess, 145.6ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 AirPod, 1 Apple, 1 CardHolder, 1 Key, 1 Mobile, 1 Orange, 1 Stapler, 1 StickyNote, 1 Watch, 150.4ms
Speed: 2.1ms preprocess, 150.4ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 AirPod, 1 Apple, 1 CardHolder, 1 Key, 1 Mobile, 1 Orange, 1 Stapler, 1 StickyNote, 1 Watch, 137.3ms
Speed: 2.1ms preprocess, 137.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 AirPod, 1 Apple, 1 CardHolder, 1 Key, 1 Mobile, 1 Orange, 1 Stapler, 1 StickyNote, 1 Watch, 138.7ms
Speed: 1.8ms preprocess, 138.7ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 AirPod, 1 Apple, 1 CardHolder, 1 Key, 1 Mobile, 1 Orange, 1 Stapler, 1 StickyNote, 1 Watch, 137.6ms
Speed: 2.3ms preprocess, 137.6ms infer

In [7]:
#Loop over all folders for distance values in csv
#Experiment final (15/04/25)

In [11]:
import cv2
import numpy as np
from ultralytics import YOLO
import os
import glob
import csv

# ✅ Load YOLOv8 model
model = YOLO('assignment_3_model_yolo11s.pt')

# ✅ Updated root directory
base_root = r"F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\Deliverable\Dataset\Before annotation"

# ✅ RealSense D455: meters
depth_scale = 0.001

# ✅ Expected CSV headers
expected_classes = [
    "apple", "watch", "airpods", "cardholder", "mouse",
    "key", "stapler", "orange", "phone", "stickynote"
]

# ✅ Mapping YOLO class names to target column names
class_name_map = {
    "Apple": "apple",
    "Watch": "watch",
    "AirPod": "airpods",
    "CardHolder": "cardholder",
    "Mouse": "mouse",
    "Key": "key",
    "Stapler": "stapler",
    "Orange": "orange",
    "Mobile": "phone",
    "StickyNote": "stickynote"
}

# ✅ Iterate through all scenarios and places
for scenario in range(1, 4):
    for place in range(1, 4):
        print(f"🔍 Processing Scenario_{scenario}, Place_{place}...")

        # Construct paths
        place_folder = os.path.join(base_root, f"Scenario_{scenario}", f"Place_{place}")
        rgb_folder = os.path.join(place_folder, "RGB")
        npy_folder = os.path.join(place_folder, "NPY")
        csv_output_path = os.path.join(place_folder, "object_distances.csv")

        # Check existence
        if not os.path.exists(rgb_folder) or not os.path.exists(npy_folder):
            print(f"⚠️ Skipping: Missing RGB or NPY folder in {place_folder}")
            continue

        # List RGB frames
        rgb_files = sorted(glob.glob(os.path.join(rgb_folder, "rgb_frame_*.png")))
        if not rgb_files:
            print(f"⚠️ No RGB images found in: {rgb_folder}")
            continue

        # Write CSV
        with open(csv_output_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["image"] + expected_classes)

            for rgb_path in rgb_files:
                index = os.path.splitext(os.path.basename(rgb_path))[0].split("_")[-1]
                depth_path = os.path.join(npy_folder, f"depth_raw_{index}.npy")

                if not os.path.exists(depth_path):
                    print(f"[Warning] Missing depth file for: {rgb_path}")
                    continue

                # Load data
                rgb_image = cv2.imread(rgb_path)
                depth_image = np.load(depth_path).astype(np.float32) * depth_scale

                # Run object detection
                results = model(rgb_image)
                object_depths = {cls: "" for cls in expected_classes}

                for result in results:
                    for box in result.boxes:
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                        confidence = box.conf[0].cpu().numpy()
                        class_id = int(box.cls[0].cpu().numpy())
                        detected_name = model.names[class_id]
                        class_name = class_name_map.get(detected_name, None)

                        if confidence < 0.5 or class_name is None:
                            continue

                        # Extract depth region
                        crop = depth_image[y1:y2, x1:x2]
                        valid_depths = crop[(crop > 0) & (crop < 10)]
                        if valid_depths.size == 0:
                            continue

                        # Save median distance (adjusted by 0.92 factor)
                        object_depths[class_name] = round(np.median(valid_depths * 0.92), 3)

                # Write one row to CSV
                row = [os.path.basename(rgb_path)] + [object_depths[cls] for cls in expected_classes]
                writer.writerow(row)

        print(f"✅ CSV saved at: {csv_output_path}")


🔍 Processing Scenario_1, Place_1...

0: 480x640 1 AirPod, 1 Apple, 1 CardHolder, 1 Key, 1 Mobile, 1 Mouse, 1 Orange, 1 Stapler, 1 StickyNote, 1 Watch, 420.0ms
Speed: 8.7ms preprocess, 420.0ms inference, 4.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 AirPod, 1 Apple, 1 CardHolder, 1 Key, 1 Mobile, 1 Mouse, 1 Orange, 1 Stapler, 1 StickyNote, 1 Watch, 398.4ms
Speed: 4.4ms preprocess, 398.4ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 AirPod, 1 Apple, 1 CardHolder, 1 Key, 1 Mobile, 1 Mouse, 1 Orange, 1 Stapler, 1 StickyNote, 1 Watch, 408.6ms
Speed: 4.1ms preprocess, 408.6ms inference, 3.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 AirPod, 1 Apple, 1 CardHolder, 1 Key, 1 Mobile, 1 Mouse, 1 Orange, 1 Stapler, 1 StickyNote, 1 Watch, 404.7ms
Speed: 4.5ms preprocess, 404.7ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 AirPod, 1 Apple, 1 CardHolder, 1 Key, 1 Mobile, 1 Mouse, 1 Orange,

In [29]:
#Analysis and Plot

In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Base directory for the dataset
base_dir = r"F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\Deliverable\Dataset\Before annotation"

# Helper function to normalize names
normalize = lambda s: s.replace(" ", "").strip().lower()

# Store scenario-wise summary of average errors
summary_errors = {}

# Loop through scenarios and places
for scenario_id in range(1, 4):
    scenario_name = f"Scenario_{scenario_id}"
    scenario_errors = []

    for place_id in range(1, 4):
        place_name = f"Place_{place_id}"
        place_dir = os.path.join(base_dir, scenario_name, place_name)

        csv_path = os.path.join(place_dir, "object_distances.csv")
        gt_path = os.path.join(place_dir, "Ground_Truth_Distance.txt")

        if not os.path.exists(csv_path) or not os.path.exists(gt_path):
            print(f"⚠️ Skipping {scenario_name}/{place_name} — Missing files.")
            continue

        # Load CSV and skip first outlier row
        df = pd.read_csv(csv_path)
        df = df.iloc[1:]  # Remove the outlier row
        df_numeric = df.iloc[:, 1:]  # Remove image column

        # Normalize CSV columns
        normalized_csv_columns = [normalize(col) for col in df_numeric.columns]
        original_column_map = {normalize(col): col for col in df_numeric.columns}

        # Read all ground truth values into dictionary
        full_true_distances = {}
        with open(gt_path, "r") as f:
            for line in f:
                if ':' in line:
                    key, val = line.strip().split(":")
                    key_clean = normalize(key)
                    val_clean = val.strip().replace(",", "")
                    try:
                        full_true_distances[key_clean] = float(val_clean)
                    except ValueError:
                        print(f"⚠️ Skipping invalid value in {gt_path}: {line.strip()}")

        # Match columns from CSV with ground truth (after normalization)
        matched_keys = [col for col in normalized_csv_columns if col in full_true_distances]
        if not matched_keys:
            print(f"⚠️ No matching object columns found in {scenario_name}/{place_name}")
            continue

        # Prepare mappings
        matched_original_columns = [original_column_map[k] for k in matched_keys]
        matched_true_values = {original_column_map[k]: full_true_distances[k] for k in matched_keys}

        # Select columns and calculate error
        df_objects = df_numeric[matched_original_columns]
        error_df = abs(df_objects - pd.Series(matched_true_values))

        # Plot per-object error
        for obj in matched_original_columns:
            plt.figure(figsize=(10, 4))
            plt.plot(error_df.index, error_df[obj], label=f"{obj}", color='blue')
            plt.title(f"{scenario_name} - {place_name} - {obj}")
            plt.xlabel("Frame Index")
            plt.ylabel("Absolute Error (meters)")
            plt.grid(True)
            plt.legend()
            plot_file = os.path.join(place_dir, f"{scenario_name}_{place_name}_{obj}.png")
            plt.savefig(plot_file)
            plt.close()

        scenario_errors.append(error_df)

    # After all 3 places in a scenario
    if scenario_errors:
        combined_error_df = pd.concat(scenario_errors, ignore_index=True)
        avg_errors = combined_error_df.mean().sort_values()
        summary_errors[scenario_name] = avg_errors

        # Bar plot
        plt.figure(figsize=(10, 5))
        avg_errors.plot(kind='bar', color='skyblue')
        plt.title(f"Average Absolute Error per Object - {scenario_name}")
        plt.ylabel("Average Error (m)")
        plt.grid(True, axis='y')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(os.path.join(base_dir, f"{scenario_name}_average_error_bar.png"))
        plt.close()

        # Heatmap
        plt.figure(figsize=(12, 6))
        sns.heatmap(combined_error_df.T, cmap="viridis", cbar_kws={'label': 'Absolute Error (m)'})
        plt.title(f"Error Heatmap - {scenario_name}")
        plt.xlabel("Frame Index")
        plt.ylabel("Object")
        plt.tight_layout()
        plt.savefig(os.path.join(base_dir, f"{scenario_name}_heatmap.png"))
        plt.close()

print("✅ All error plots and summaries saved successfully.")


⚠️ No matching object columns found in Scenario_1/Place_1
⚠️ No matching object columns found in Scenario_1/Place_2
⚠️ No matching object columns found in Scenario_1/Place_3
⚠️ No matching object columns found in Scenario_2/Place_1
⚠️ No matching object columns found in Scenario_2/Place_2
⚠️ No matching object columns found in Scenario_2/Place_3
⚠️ No matching object columns found in Scenario_3/Place_1
⚠️ No matching object columns found in Scenario_3/Place_2
⚠️ No matching object columns found in Scenario_3/Place_3
✅ All error plots and summaries saved successfully.


In [31]:
import pandas as pd
import os

# Set directory and file paths
place_dir = r"F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\Deliverable\Dataset\Before annotation\Scenario_1\Place_1"
csv_path = os.path.join(place_dir, "object_distances.csv")
gt_path = os.path.join(place_dir, "Ground_Truth_Distance.txt")

# Read CSV and ignore first column
df = pd.read_csv(csv_path)
csv_columns = df.columns[1:]  # Skip 'image' column

print("📄 CSV Columns (excluding first column):")
print([col.strip() for col in csv_columns])

# Read ground truth keys, remove single quotes
gt_keys = []
with open(gt_path, "r") as f:
    for line in f:
        if ':' in line:
            key, _ = line.strip().split(":")
            key_clean = key.strip().replace("'", "")  # Remove single quotes
            gt_keys.append(key_clean)

print("\n📄 Ground Truth Keys (cleaned):")
print(gt_keys)


📄 CSV Columns (excluding first column):
['apple', 'watch', 'airpods', 'cardholder', 'mouse', 'key', 'stapler', 'orange', 'phone', 'stickynote']

📄 Ground Truth Keys (cleaned):
['apple', 'watch', 'airpods', 'cardholder', 'mouse', 'key', 'stapler', 'orange', 'phone', 'stickynote']


In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Base directory
base_dir = r"F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\Deliverable\Dataset\Before annotation"

# Normalization helper
normalize = lambda s: s.replace(" ", "").replace("'", "").strip().lower()

# Dictionary to accumulate errors per scenario-object
scenario_object_errors = {}

# Loop through scenarios
for scenario_id in range(1, 4):
    scenario_name = f"Scenario_{scenario_id}"
    scenario_object_errors[scenario_name] = {}

    for place_id in range(1, 4):
        place_name = f"Place_{place_id}"
        place_dir = os.path.join(base_dir, scenario_name, place_name)

        csv_path = os.path.join(place_dir, "object_distances.csv")
        gt_path = os.path.join(place_dir, "Ground_Truth_Distance.txt")

        if not os.path.exists(csv_path) or not os.path.exists(gt_path):
            print(f"⚠️ Skipping {scenario_name}/{place_name} — Missing files.")
            continue

        # Load CSV, drop first row, and ignore first column
        df = pd.read_csv(csv_path)
        df = df.iloc[1:]  # Remove possible outlier
        df_numeric = df.iloc[:, 1:]

        # Normalize CSV columns
        original_columns = list(df_numeric.columns)
        normalized_csv_map = {normalize(col): col for col in original_columns}

        # Read and normalize ground truth
        true_distances = {}
        with open(gt_path, "r") as f:
            for line in f:
                if ':' in line:
                    key, val = line.strip().split(":")
                    key_clean = normalize(key)
                    val_clean = val.strip().replace(",", "")
                    try:
                        true_distances[key_clean] = float(val_clean)
                    except ValueError:
                        print(f"⚠️ Invalid value in GT: {line.strip()}")

        # Match columns in CSV with ground truth
        matched_keys = [k for k in normalized_csv_map if k in true_distances]
        matched_columns = [normalized_csv_map[k] for k in matched_keys]

        if not matched_columns:
            print(f"⚠️ No matching object columns found in {scenario_name}/{place_name}")
            continue

        df_objects = df_numeric[matched_columns]
        matched_true_values = {normalized_csv_map[k]: true_distances[k] for k in matched_keys}
        error_df = abs(df_objects - pd.Series(matched_true_values))

        # Store errors grouped by scenario and object
        for obj in matched_columns:
            if obj not in scenario_object_errors[scenario_name]:
                scenario_object_errors[scenario_name][obj] = []
            scenario_object_errors[scenario_name][obj].extend(error_df[obj].tolist())

# 🔧 Now generate plots for each object in each scenario
for scenario_name, object_errors in scenario_object_errors.items():
    for obj_name, error_values in object_errors.items():
        plt.figure(figsize=(10, 4))
        plt.plot(range(len(error_values)), error_values, label=obj_name, color='blue')
        plt.title(f"{scenario_name} - {obj_name} - Distance Error")
        plt.xlabel("Frame Index")
        plt.ylabel("Absolute Error (m)")
        plt.grid(True)
        plt.legend()
        output_plot_path = os.path.join(base_dir, f"{scenario_name}_{obj_name}_error_plot.png")
        plt.tight_layout()
        plt.savefig(output_plot_path)
        plt.close()

print("✅ All error plots generated successfully.")


✅ All error plots generated successfully.


In [45]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from collections import defaultdict

# Base directory for dataset
base_dir = r"F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\Deliverable\Dataset\Before annotation"

# Normalization helper
normalize = lambda s: s.replace(" ", "").replace("'", "").strip().lower()

# To store scenario-level aggregation
scenario_object_errors = {}

# Loop through each scenario
for scenario_id in range(1, 4):
    scenario_name = f"Scenario_{scenario_id}"
    scenario_object_errors[scenario_name] = {}

    for place_id in range(1, 4):
        place_name = f"Place_{place_id}"
        place_dir = os.path.join(base_dir, scenario_name, place_name)

        csv_path = os.path.join(place_dir, "object_distances.csv")
        gt_path = os.path.join(place_dir, "Ground_Truth_Distance.txt")

        if not os.path.exists(csv_path) or not os.path.exists(gt_path):
            print(f"⚠️ Skipping {scenario_name}/{place_name} — Missing files.")
            continue

        # Load and clean CSV
        df = pd.read_csv(csv_path)
        df = df.iloc[1:]  # Remove potential outlier
        df_numeric = df.iloc[:, 1:]  # Ignore image column

        # Normalize CSV column names
        original_columns = list(df_numeric.columns)
        normalized_csv_map = {normalize(col): col for col in original_columns}

        # Read and normalize ground truth
        true_distances = {}
        with open(gt_path, "r") as f:
            for line in f:
                if ':' in line:
                    key, val = line.strip().split(":")
                    key_clean = normalize(key)
                    val_clean = val.strip().replace(",", "")
                    try:
                        true_distances[key_clean] = float(val_clean)
                    except ValueError:
                        print(f"⚠️ Invalid value in GT: {line.strip()}")

        # Match columns
        matched_keys = [k for k in normalized_csv_map if k in true_distances]
        matched_columns = [normalized_csv_map[k] for k in matched_keys]

        if not matched_columns:
            print(f"⚠️ No matching object columns found in {scenario_name}/{place_name}")
            continue

        df_objects = df_numeric[matched_columns]
        matched_true_values = {normalized_csv_map[k]: true_distances[k] for k in matched_keys}
        error_df = abs(df_objects - pd.Series(matched_true_values))

        # 🔹 Save object-wise plots per place
        for obj in matched_columns:
            error_values = error_df[obj].tolist()
            plt.figure(figsize=(10, 4))
            plt.plot(range(len(error_values)), error_values, label=obj, color='green')
            plt.title(f"{scenario_name} - {place_name} - {obj} Distance Error")
            plt.xlabel("Frame Index")
            plt.ylabel("Absolute Error (m)")
            plt.grid(True)
            plt.legend()
            plt.tight_layout()

            plot_path = os.path.join(place_dir, f"{scenario_name}_{place_name}_{obj}_error_plot.png")
            plt.savefig(plot_path)
            plt.close()

        # 🔹 Accumulate errors for scenario-level combination
        for obj in matched_columns:
            if obj not in scenario_object_errors[scenario_name]:
                scenario_object_errors[scenario_name][obj] = []
            scenario_object_errors[scenario_name][obj].extend(error_df[obj].tolist())

# ✅ Combined Plot for Each Object Across Scenarios with Custom Labels
# Rearranging for object -> scenario
object_scenario_errors = defaultdict(dict)

for scenario_name, obj_errors in scenario_object_errors.items():
    for obj_name, error_list in obj_errors.items():
        object_scenario_errors[obj_name][scenario_name] = error_list

# Custom legend labels
legend_labels = {
    "Scenario_1": "Frosted Glass",
    "Scenario_2": "Transparent Glass",
    "Scenario_3": "No Glass"
}

# Plot combined object-wise across scenarios
for obj_name, scenario_data in object_scenario_errors.items():
    plt.figure(figsize=(12, 5))

    for scenario_name, error_values in scenario_data.items():
        label = legend_labels.get(scenario_name, scenario_name)
        plt.plot(range(len(error_values)), error_values, label=label)

    plt.title(f"Distance Error for {obj_name} Across Scenarios")
    plt.xlabel("Frame Index")
    plt.ylabel("Absolute Error (m)")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()

    combined_plot_path = os.path.join(base_dir, f"{obj_name}_combined_error_plot.png")
    plt.savefig(combined_plot_path)
    plt.close()

print("✅ Place-wise and scenario-wise error plots generated successfully.")


✅ Place-wise and scenario-wise error plots generated successfully.
